# **Marble Solitaire by using Priority Queue**

In [ ]:
import heapq
import copy
import time

SIZE = 7

class Node:
    def __init__(self, board, cost, path):
        self.board = board
        self.cost = cost
        self.path = path

    def __lt__(self, other):
        return self.cost < other.cost

# Initial configuration of the board
initial_board = [
    [-1, -1, 1, 1, 1, -1, -1],
    [-1, -1, 1, 1, 1, -1, -1],
    [1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1],
    [-1, -1, 1, 1, 1, -1, -1],
    [-1, -1, 1, 1, 1, -1, -1]
]

MOVES = [(-2, 0), (2, 0), (0, -2), (0, 2)]  # Up, down, left, right

def is_valid_move(board, x, y, dx, dy):
    nx, ny = x + dx, y + dy
    mx, my = x + dx // 2, y + dy // 2
    return (0 <= nx < SIZE and 0 <= ny < SIZE and
            board[nx][ny] == 0 and board[mx][my] == 1 and board[x][y] == 1)

def apply_move(board, x, y, dx, dy):
    new_board = copy.deepcopy(board)
    nx, ny = x + dx, y + dy
    mx, my = x + dx // 2, y + dy // 2
    new_board[x][y], new_board[mx][my], new_board[nx][ny] = 0, 0, 1
    return new_board

def goal_test(board):
    return board[3][3] == 1 and sum(row.count(1) for row in board) == 1

def path_cost(board):
    return sum(row.count(1) for row in board)

def board_to_tuple(board):
    return tuple(tuple(row) for row in board)

def priority_queue_search(initial_board):
    pq = []
    visited = set()  # To track visited states
    initial_node = Node(initial_board, path_cost(initial_board), [])
    heapq.heappush(pq, initial_node)

    while pq:
        current_node = heapq.heappop(pq)

        if goal_test(current_node.board):
            return current_node.path  # Return the sequence of moves

        current_board_tuple = board_to_tuple(current_node.board)
        if current_board_tuple in visited:
            continue
        visited.add(current_board_tuple)  # Mark the state as visited

        # Explore valid moves
        for i in range(SIZE):
            for j in range(SIZE):
                if current_node.board[i][j] == 1:
                    for dx, dy in MOVES:
                        if is_valid_move(current_node.board, i, j, dx, dy):
                            new_board = apply_move(current_node.board, i, j, dx, dy)
                            new_cost = path_cost(new_board)
                            new_node = Node(new_board, new_cost, current_node.path + [(i, j, dx, dy)])
                            heapq.heappush(pq, new_node)

    return None

start_time = time.time()

result = priority_queue_search(initial_board)

end_time = time.time()

time_taken = end_time - start_time

if result:
    print("Solution found :")
    for step in result:
        print(step)
else:
    print("No solution found.")

print(f"Time taken: {time_taken:.4f} seconds")

# Solution is of type (x,y,dx,dy)
# where (x,y) :  The Coordinates of the Marble being moved.
#       (dx,dy) : The Direction and Distance of the move.
# So below Solution is the Sequence of States to Reach Goal State (3,3) that is Center of The Board

Solution found :
(1, 3, 2, 0)
(2, 1, 0, 2)
(0, 2, 2, 0)
(0, 4, 0, -2)
(2, 3, 0, -2)
(2, 0, 0, 2)
(2, 4, -2, 0)
(2, 6, 0, -2)
(3, 2, -2, 0)
(0, 2, 2, 0)
(3, 0, 0, 2)
(3, 2, -2, 0)
(3, 4, -2, 0)
(0, 4, 2, 0)
(3, 6, 0, -2)
(3, 4, -2, 0)
(5, 2, -2, 0)
(4, 0, 0, 2)
(4, 2, -2, 0)
(1, 2, 2, 0)
(3, 2, 0, 2)
(4, 4, -2, 0)
(1, 4, 2, 0)
(4, 6, 0, -2)
(4, 3, 0, 2)
(6, 4, -2, 0)
(3, 4, 2, 0)
(6, 2, 0, 2)
(6, 4, -2, 0)
(4, 5, 0, -2)
(5, 3, -2, 0)
Time taken: 1.7956 seconds


In [ ]:
# Heuristic 1: Number of remaining marbles

def h1(board):
    return sum(row.count(1) for row in board)

In [ ]:
# Heuristic 2: Distance of marbles from the center

def h2(board):
    total_distance = 0
    for i in range(SIZE):
        for j in range(SIZE):
            if board[i][j] == 1:
                total_distance += abs(i - 3) + abs(j - 3)
    return total_distance

# **Marble Solitaire by using Best First Search**

In [ ]:
def best_first_search(initial_board, heuristic):
    pq = []
    visited = set()
    initial_node = Node(initial_board, heuristic(initial_board), [])
    heapq.heappush(pq, initial_node)

    while pq:
        current_node = heapq.heappop(pq)

        # Check if we reached the goal
        if goal_test(current_node.board):
            return current_node.path

        # Mark the board as visited
        board_tuple = board_to_tuple(current_node.board)
        if board_tuple in visited:
            continue
        visited.add(board_tuple)

        # Explore valid moves
        for i in range(SIZE):
            for j in range(SIZE):
                if current_node.board[i][j] == 1:
                    for dx, dy in MOVES:
                        if is_valid_move(current_node.board, i, j, dx, dy):
                            new_board = apply_move(current_node.board, i, j, dx, dy)
                            if board_to_tuple(new_board) not in visited:
                                new_node = Node(new_board, heuristic(new_board), current_node.path + [(i, j, dx, dy)])
                                heapq.heappush(pq, new_node)

    return None

# Run Best First Search with Heuristic 1
start_time = time.time()

result = best_first_search(initial_board,h1)

end_time = time.time()

time_taken = end_time - start_time
if result:
    print("Solution found with Best First Search (h1) :")
    for step in result:
        print(step)
else:
    print("No solution found.")

print(f"Time taken: {time_taken:.4f} seconds")

Solution found with Best First Search (h1) :
(1, 3, 2, 0)
(2, 1, 0, 2)
(0, 2, 2, 0)
(0, 4, 0, -2)
(2, 3, 0, -2)
(2, 0, 0, 2)
(2, 4, -2, 0)
(2, 6, 0, -2)
(3, 2, -2, 0)
(0, 2, 2, 0)
(3, 0, 0, 2)
(3, 2, -2, 0)
(3, 4, -2, 0)
(0, 4, 2, 0)
(3, 6, 0, -2)
(3, 4, -2, 0)
(5, 2, -2, 0)
(4, 0, 0, 2)
(4, 2, -2, 0)
(1, 2, 2, 0)
(3, 2, 0, 2)
(4, 4, -2, 0)
(1, 4, 2, 0)
(4, 6, 0, -2)
(4, 3, 0, 2)
(6, 4, -2, 0)
(3, 4, 2, 0)
(6, 2, 0, 2)
(6, 4, -2, 0)
(4, 5, 0, -2)
(5, 3, -2, 0)
Time taken: 1.5281 seconds


In [ ]:
def best_first_search(initial_board, heuristic):
    pq = []
    visited = set()
    initial_node = Node(initial_board, heuristic(initial_board), [])
    heapq.heappush(pq, initial_node)

    while pq:
        current_node = heapq.heappop(pq)

        # Check if we reached the goal
        if goal_test(current_node.board):
            return current_node.path

        # Mark the board as visited
        board_tuple = board_to_tuple(current_node.board)
        if board_tuple in visited:
            continue
        visited.add(board_tuple)

        # Explore valid moves
        for i in range(SIZE):
            for j in range(SIZE):
                if current_node.board[i][j] == 1:
                    for dx, dy in MOVES:
                        if is_valid_move(current_node.board, i, j, dx, dy):
                            new_board = apply_move(current_node.board, i, j, dx, dy)
                            if board_to_tuple(new_board) not in visited:
                                new_node = Node(new_board, heuristic(new_board), current_node.path + [(i, j, dx, dy)])
                                heapq.heappush(pq, new_node)

    return None

# Run Best First Search with Heuristic 2
start_time = time.time()

result = best_first_search(initial_board,h2)

end_time = time.time()

time_taken = end_time - start_time
if result:
    print("Solution found with Best First Search (h2) :")
    for step in result:
        print(step)
else:
    print("No solution found.")

print(f"Time taken: {time_taken:.4f} seconds")

Solution found with Best First Search (h2) :
(1, 3, 2, 0)
(2, 1, 0, 2)
(0, 2, 2, 0)
(0, 4, 0, -2)
(4, 1, -2, 0)
(2, 4, -2, 0)
(2, 6, 0, -2)
(4, 6, -2, 0)
(4, 5, -2, 0)
(3, 2, -2, 0)
(0, 2, 2, 0)
(5, 2, -2, 0)
(3, 4, -2, 0)
(0, 4, 2, 0)
(4, 3, 0, 2)
(6, 4, -2, 0)
(6, 2, 0, 2)
(3, 2, -2, 0)
(2, 0, 0, 2)
(4, 0, -2, 0)
(2, 3, 0, -2)
(2, 0, 0, 2)
(1, 2, 2, 0)
(3, 2, 0, 2)
(3, 4, -2, 0)
(2, 6, 0, -2)
(1, 4, 2, 0)
(3, 4, 2, 0)
(6, 4, -2, 0)
(4, 5, 0, -2)
(5, 3, -2, 0)
Time taken: 30.0608 seconds


# **Marble Solitaire by using A Star**

In [ ]:
def a_star_search(initial_board, heuristic):
    pq = []
    visited = set()
    initial_node = Node(initial_board, heuristic(initial_board), [])
    heapq.heappush(pq, initial_node)
    visited.add(board_to_tuple(initial_board))

    while pq:
        current_node = heapq.heappop(pq)

        # Check if we reached the goal state
        if goal_test(current_node.board):
            return list(reversed(current_node.path))

        # Explore all possible moves from the current node
        for i in range(SIZE):
            for j in range(SIZE):
                if current_node.board[i][j] == 1:
                    for dx, dy in MOVES:
                        if is_valid_move(current_node.board, i, j, dx, dy):
                            new_board = apply_move(current_node.board, i, j, dx, dy)
                            board_tuple = board_to_tuple(new_board)

                            if board_tuple not in visited:
                                new_cost = len(current_node.path) + 1
                                total_cost = new_cost + heuristic(new_board)
                                new_node = Node(new_board, total_cost, current_node.path + [(i, j, dx, dy)])
                                heapq.heappush(pq, new_node)
                                visited.add(board_tuple)

    return None

# Run A* Search with h1
start_time = time.time()

result = a_star_search(initial_board,h1)

end_time = time.time()

time_taken = end_time - start_time
if result:
    print("Solution found with A* (h1) :")
    for step in result:
        print(step)
else:
    print("No solution found.")

print(f"Time taken: {time_taken:.4f} seconds")

Solution found with A* (h1) :
(1, 3, 2, 0)
(2, 5, 0, -2)
(0, 4, 2, 0)
(0, 2, 0, 2)
(3, 4, -2, 0)
(0, 4, 2, 0)
(2, 3, 0, 2)
(2, 1, 0, 2)
(4, 2, -2, 0)
(2, 3, 0, -2)
(2, 0, 0, 2)
(1, 2, 2, 0)
(2, 6, 0, -2)
(5, 4, -2, 0)
(2, 4, 2, 0)
(6, 2, -2, 0)
(3, 2, 2, 0)
(3, 0, 0, 2)
(3, 2, 0, 2)
(3, 4, 2, 0)
(3, 6, 0, -2)
(6, 4, -2, 0)
(3, 4, 2, 0)
(4, 0, 0, 2)
(4, 2, 2, 0)
(4, 6, 0, -2)
(4, 3, 0, 2)
(6, 2, 0, 2)
(6, 4, -2, 0)
(4, 5, 0, -2)
(5, 3, -2, 0)
Time taken: 0.3846 seconds


In [ ]:
def a_star_search(initial_board, heuristic):
    pq = []
    visited = set()
    initial_node = Node(initial_board, heuristic(initial_board), [])
    heapq.heappush(pq, initial_node)
    visited.add(board_to_tuple(initial_board))

    while pq:
        current_node = heapq.heappop(pq)

        # Check if we reached the goal state
        if goal_test(current_node.board):
            return (current_node.path)

        # Explore all possible moves from the current node
        for i in range(SIZE):
            for j in range(SIZE):
                if current_node.board[i][j] == 1:
                    for dx, dy in MOVES:
                        if is_valid_move(current_node.board, i, j, dx, dy):
                            new_board = apply_move(current_node.board, i, j, dx, dy)
                            board_tuple = board_to_tuple(new_board)

                            if board_tuple not in visited:
                                new_cost = len(current_node.path) + 1
                                total_cost = new_cost + heuristic(new_board)
                                new_node = Node(new_board, total_cost, current_node.path + [(i, j, dx, dy)])
                                heapq.heappush(pq, new_node)
                                visited.add(board_tuple)

    return None

# Run A* Search with h2
start_time = time.time()

result = a_star_search(initial_board,h2)

end_time = time.time()

time_taken = end_time - start_time
if result:
    print("Solution found with A* (h2) :")
    for step in result:
        print(step)
else:
    print("No solution found.")

print(f"Time taken: {time_taken:.4f} seconds")

Solution found with A* (h2) :
(1, 3, 2, 0)
(2, 1, 0, 2)
(0, 2, 2, 0)
(0, 4, 0, -2)
(4, 1, -2, 0)
(2, 4, -2, 0)
(2, 6, 0, -2)
(4, 6, -2, 0)
(4, 5, -2, 0)
(3, 4, -2, 0)
(0, 4, 2, 0)
(5, 4, -2, 0)
(3, 2, -2, 0)
(2, 0, 0, 2)
(4, 0, -2, 0)
(4, 3, 0, -2)
(6, 2, -2, 0)
(6, 4, 0, -2)
(2, 3, 0, -2)
(0, 2, 2, 0)
(3, 4, -2, 0)
(2, 6, 0, -2)
(1, 4, 2, 0)
(3, 4, 0, -2)
(3, 2, -2, 0)
(2, 0, 0, 2)
(1, 2, 2, 0)
(3, 2, 2, 0)
(6, 2, -2, 0)
(4, 1, 0, 2)
(5, 3, -2, 0)
Time taken: 156.3677 seconds
